In [1]:
import pandas as pd
import numpy as np

#UTILIZAR FUNÇÕES PARA LIMPAR CW E WS OK
#FAZER 'UNION' DE TODOS OS ARQUIVOS COMO NA BASE_IMPORT_ARIBA
#CORRIGIR CÁLCULO DE PERCENTUAL OPEX DA BASE SR

In [2]:
df_ariba_cw = pd.read_csv('Contrato_Legal_Compras.csv', sep=',', encoding='latin1')
df_ariba_ws = pd.read_csv('Projeto_de_Sourcing.csv', sep=',', encoding='latin1')
df_ariba_sr = pd.read_csv('Solicitação_Sourcing.csv', sep = ',', encoding = 'latin1')
#df_ariba_sr.head()
#df_ariba_cw.head()
#df_ariba_ws.head()

In [3]:
df_contas_third_parties = pd.read_excel('DEPARA_CONTAS_THIRD.xlsx')
#df_contas_third_parties.head()

In [4]:
df_de_para_uo_cdc = pd.read_excel('DE_PARA_UO_CDC.xlsx')
df_de_para_uo_cdc.head()

,N1,N2,N3,DETALHAMENTO,CDC
0,Audit,Audit,Audit,Audit,FAHR000
1,Audit,"Planning, Forensic & Special Audit Proj.","Planning, Forensic & Special Audit Proj.","Planning, Forensic & Special Audit Proj.",FAHR000
2,Audit,Technical Audit,Technical Audit,Technical Audit,FAHR000
3,Business Support Officer,Business Support Officer,Business Support Officer,Business Support Officer,HSHR500
4,Business Support Officer,Business Support Processes & Reporting,Business Support Processes & Reporting,Business Support Processes & Reporting,HSHR010


In [5]:
#new_columns_names = ['NomeArquivo1', 'NomeArquivo2', 'NomeArquivo3', 'NomeArquivo4', 'NomeArquivo5']

#FUNÇÕES PARA LIMPEZA DOS DFS

def create_column_three(df, column_1, new_column): #column_3, column_4,
    df[new_column] = (
        df[column_1].astype(str)) #df[column_3].astype(str) + " - " df[column_4].astype(str)
    return df

def remove_columns(df, columns_names): #vai receber um array de colunas ou uma só coluna
    df = df.drop(columns = columns_names, axis = 1)
    return df

def remove_columns_condicional(df, columns_names): #vai receber um array de colunas ou uma só coluna e remover cujo 'STATUS' for CANCELADO
    df = df[df[columns_names] != 'Cancelado'] 
    return df

def rename_columns(df, columns_names):
    df = df.rename(columns = columns_names) #vai receber um 'dicionário' de colunas ou uma única coluna 
    return df

def replace_values(df, column_name):
    df[column_name] = df[column_name].replace('Irrestrito', '', regex = False)
    return df

def transform_column_type(df, column_name):
    for col in column_name:
        df[col] = (df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float).round(2))
    return df

def transform_column_int (df, column_name):
    df[column_name] = (df[column_name].astype(int))
    return df

def split_column(df, to_split_column, new_columns):
    df[new_columns] = df[to_split_column].str.split('_', expand = True)
    return df

def split_column_2(df, to_split_column, new_columns):
    df[new_columns] = df[to_split_column].str.split('-', n=len(new_columns)-1, expand=True)
    return df

def transform_column_text(df, column_name):
    for col in column_name:
        df[col] = (df[col].astype(str))
    return df
    
def transform_column_date_1 (df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    return df

def transform_column_date(df, column_names):
    for col in column_names:
        df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')
    return df

def sort_ascending(df, column_name):
    df.sort_values(by = column_name)
    return df

def reorder_columns(df, new_order):
    df = df.loc[:, new_order]
    return df

def clean_cnpj (df, column_name):
    df[column_name] = (df[column_name].astype(str).str.replace
                                       ('.', '', regex=False).str.replace('-', '.', regex=False).str.replace
                                       ('/', '', regex=False).str.replace("'", "", regex=False))
    return df

def sort_values(df, column_name):
    df = df.sort_values(by= column_name)
    return df

def valid_value(df, column_name):
    df= df[df[column_name]>0]
    return df

def order_date(df, column_name):
    last_date = df[column_name].max() 	
    df = df[df[column_name] == last_date]
    return df

In [6]:
#COMEÇANDO A LIMPEZA DO DF CW:
to_remove_4 = ['Contrato - Data de efetivaÃ§Ã£o', 'Contrato - Data de vencimento', 'Status do contrato', 'count()']
df_ariba_cw = remove_columns(df_ariba_cw, to_remove_4)

to_transform_type_2 = ['sum(Valor do contrato)']
df_ariba_cw = transform_column_type(df_ariba_cw, to_transform_type_2) 

new_columns__3 = ['NomeArquivo1', 'NomeArquivo2', 'NomeArquivo3', 'NomeArquivo4', 'NomeArquivo5']
to_split_column_3 = 'NomeArquivo'
df_ariba_cw = split_column(df_ariba_cw, to_split_column_3, new_columns__3) 

to_remove_5 = ['NomeArquivo','NomeArquivo2','NomeArquivo3','NomeArquivo4','NomeArquivo5']
df_ariba_cw = remove_columns(df_ariba_cw, to_remove_5) 

to_rename_3 = {'NomeArquivo1': 'DATA_REFERENCIA'}
df_ariba_cw = rename_columns (df_ariba_cw, to_rename_3)

column_name = 'CNPJ/CPF do Fornecedor'
df_ariba_cw = clean_cnpj(df_ariba_cw, column_name) 

to_rename_4 = {'Baseado no projeto... - CÃ³digo do projeto': 'BASEADO_NO_PROJ', 
               'CÃ³digo do projeto': 'CW', 
               'CNPJ/CPF do Fornecedor':'CNPJ_CPF_FORNEC', 
               'Fornecedor - Fornecedor global': 'FORNECEDOR', 
               'sum(Valor do contrato)': 'VALOR_CONTRATO', 
               'Contrato-raiz - CÃ³digo do projeto': 'CW_RAIZ'}
df_ariba_cw = rename_columns(df_ariba_cw, to_rename_4) 

column_name = 'BASEADO_NO_PROJ'
df_ariba_cw = replace_values(df_ariba_cw, column_name) 

#CRIAR COLUNAS SR E CW PARA VALORES QUE COMECEM COM SR OU CW NA COLUNA BASEADO_NO_PROJ
df_ariba_cw['BASEADO_NO_PROJ'] = df_ariba_cw['BASEADO_NO_PROJ'].astype(str) #TRANSOFROMA EM STRING
#SUBSTITUI POR VAZIO E CRIA A COLUNA NOVA COM O VALOR ENCONTRADO EM BASEADO_NO_PROJ, USA NUMPY!
df_ariba_cw['SR'] = np.where(df_ariba_cw['BASEADO_NO_PROJ'].str.startswith('SR'), df_ariba_cw['BASEADO_NO_PROJ'],'' ) 
df_ariba_cw['WS'] = np.where(df_ariba_cw['BASEADO_NO_PROJ'].str.startswith('WS'), df_ariba_cw['BASEADO_NO_PROJ'],'' )

column_name = 'CW'
df_ariba_cw = sort_values(df_ariba_cw, column_name)

#SELECIONANDO OS CONTRATOS COM VALOR ACIMA DE 0!
column_name = 'VALOR_CONTRATO'
df_ariba_cw = valid_value (df_ariba_cw, column_name) 

column_name = 'DATA_REFERENCIA'
df_ariba_cw = transform_column_date_1(df_ariba_cw, column_name) 

to_remove_6 = 'BASEADO_NO_PROJ'
df_ariba_cw = remove_columns(df_ariba_cw, to_remove_6) 

column_name = 'DATA_REFERENCIA'
df_ariba_cw = order_date(df_ariba_cw, column_name) 

column_name = 'CW_RAIZ'
df_ariba_cw = replace_values(df_ariba_cw, column_name) 
 
to_reorder_2 = ['CW', 'CW_RAIZ', 'CNPJ_CPF_FORNEC', 'FORNECEDOR', 'VALOR_CONTRATO', 'DATA_REFERENCIA', 'SR', 'WS']
df_ariba_cw = reorder_columns(df_ariba_cw, to_reorder_2)

#df_ariba_cw.tail(10)
#df_ariba_cw.info()

In [7]:
#COMEÇANDO A LIMPEZA DO DF WS:

to_remove_7 = ['[PSO] Area do comprador', 'count(SourcingProject)']
df_ariba_ws = remove_columns(df_ariba_ws, to_remove_7) 

to_replace_values_2 = ['[PSO]Data inicial (Data)', '[PSO]Data final (Data)', 
                       '[PSO]Baseado no projeto... (CÃ³digo do projeto)', 
                       '[PSO]Projeto raiz (CÃ³digo do projeto)', 
                       '[New FormulÃ¡rio de AprovaÃ§Ã£o]Fornecedor Selecionado (Nome do fornecedor (L1))']
df_ariba_ws = replace_values (df_ariba_ws, to_replace_values_2) 

to_rename_5 = {'sum(Valor Valor Negociado Ano 1)':'VALOR_ANO_1', 
                'sum(Valor Valor Negociado Ano 2)':'VALOR_ANO_2',
                'sum(Valor Valor Negociado Demais Anos)':'VALOR_DEMAIS_ANOS',
                'sum(Valor Valor Negociado Total)':'VALOR_TOTAL'}
df_ariba_ws = rename_columns (df_ariba_ws, to_rename_5)

to_transform_type_3 = ['VALOR_TOTAL', 'VALOR_DEMAIS_ANOS', 'VALOR_ANO_2', 'VALOR_ANO_1']
df_ariba_ws = transform_column_type (df_ariba_ws, to_transform_type_3) 

new_columns_4 = ['NomeArquivo1', 'NomeArquivo2', 'NomeArquivo3', 'NomeArquivo4', 'NomeArquivo5']
to_split_column_4 = 'NomeArquivo'
df_ariba_ws = split_column (df_ariba_ws, to_split_column_4, new_columns_4) 

to_remove_8 = ['NomeArquivo','NomeArquivo2','NomeArquivo3','NomeArquivo4','NomeArquivo5']
df_ariba_ws = remove_columns(df_ariba_ws, to_remove_8) 

to_rename_6 = {'NomeArquivo1':'DATA_REFERENCIA', 
                '[New FormulÃ¡rio de AprovaÃ§Ã£o]Fornecedor Selecionado (Nome do fornecedor (L1))' :'NOME_FORNECEDOR', 
                '[PSO] Unidade organizacional do comprador' : "UO_COMPRADOR", 
                '[PSO]Data inicial (Data)' : 'DATA_INI', 
                '[PSO]Data final (Data)' : 'DATA_FIM', 
                '[PSO]Projeto (CÃ³digo do projeto)' : 'WS', 
                '[PSO]Baseado no projeto... (CÃ³digo do projeto)' : 'BASEADO_NO_PROJ', 
                '[PSO]Projeto raiz (CÃ³digo do projeto)' : 'PROJ_RAIZ'}
df_ariba_ws = rename_columns (df_ariba_ws, to_rename_6) 
 
#ATUALIZA A COLUNA SR COMPARANDO COM BASEADO_NO_PROJETO E PROJETO RAIZ
df_ariba_ws['SR'] = np.where(
    df_ariba_ws['BASEADO_NO_PROJ'] == df_ariba_ws['PROJ_RAIZ'], df_ariba_ws['PROJ_RAIZ'], #Se os dois valores forem iguais, o valor de 'SR' será o valor de 'PROJ_RAIZ'
    np.where(
        (df_ariba_ws['BASEADO_NO_PROJ'] == '') & (df_ariba_ws['PROJ_RAIZ'] != ''), df_ariba_ws['PROJ_RAIZ'], #Se 'BASEADO_NO_PROJ' estiver vazio e 'PROJ_RAIZ' tiver valor, então 'SR' recebe o valor de 'PROJ_RAIZ'
        np.where(
            (df_ariba_ws['PROJ_RAIZ'] == '') & (df_ariba_ws['BASEADO_NO_PROJ'] != ""), df_ariba_ws['BASEADO_NO_PROJ'], #Se 'PROJ_RAIZ' estiver vazio e 'BASEADO_NO_PROJ' tiver valor, então 'SR' recebe o valor de 'BASEADO_NO_PROJ'
            np.where(
                (df_ariba_ws['BASEADO_NO_PROJ'] != df_ariba_ws['PROJ_RAIZ']) & #Se baseado no projeto for diferente de projeto raiz
                (df_ariba_ws['PROJ_RAIZ'] != '') & #e proj raiz for diferente de vazio, ou seja, se tiver valor
                (df_ariba_ws['BASEADO_NO_PROJ'] != ''), df_ariba_ws['PROJ_RAIZ'], # e baseado no proj diferente de vazio, se tiver valor
                '')))) #então substitui proj raiz por vazio

column_names = ['DATA_INI', 'DATA_FIM']
df_ariba_ws = transform_column_date (df_ariba_ws, column_names) 

to_remove_9 = ['BASEADO_NO_PROJ', 'PROJ_RAIZ']
df_ariba_ws = remove_columns (df_ariba_ws, to_remove_9) 

to_transform_type_date_2 = 'DATA_REFERENCIA'
df_ariba_ws = transform_column_date_1 (df_ariba_ws, to_transform_type_date_2) 

to_reorder_3 = ['WS', 'SR', 'DATA_INI', 
                'DATA_FIM', 'UO_COMPRADOR',
                'NOME_FORNECEDOR', 'VALOR_ANO_1', 
                'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS', 
                'VALOR_TOTAL', 'DATA_REFERENCIA']
df_ariba_ws = reorder_columns (df_ariba_ws, to_reorder_3) 

column_name = 'WS'
df_ariba_ws = sort_values(df_ariba_ws, column_name) 

column_name = 'DATA_REFERENCIA'
df_ariba_ws = order_date(df_ariba_ws, column_name) 

#df_ariba_ws.head()
#df_ariba_ws.info()

In [8]:
#APLICAÇÃO DAS FUNÇÕES NA LIMPEZA DO DF SR
to_remove_1 = ['[SPRO]Mercadoria (Mercadoria)', '[SPRO] Area do requisitante', 
               '[SPRO] Status do Processo', 'sum(Gastos da linha de base)', 
               '[SPRO] Tipo de Contratacao', '[FormulÃ¡rio de Sourcing (SR)] Ano de Referencia',
               'count(SourcingRequest)']
df_ariba_sr = remove_columns(df_ariba_sr, to_remove_1)

to_rename_1 = {'[SPRO]Mercadoria (CÃ³digo da mercadoria)': 'GRUPO_MERCADORIA', 
               '[SPRO] Diretoria do requisitante' : 'DIRETORIA', 
               '[SPRO] Estado' : 'STATUS', 
               '[SPRO]Data inicial (Data)' : 'DT_INI', 
               '[SPRO]Data final (Data)' : 'DT_FIM',
               '[SPRO] Unidade organizacional do requisitante' : 'UO_REQUISITANTE', 
               '[SPRO] Moeda' : 'MOEDA',
               '[FormulÃ¡rio de Sourcing (SR)] Conta ContÃ¡bil' : 'CONTA_CONTABIL',
               'sum(Valor OrÃ§amento Total Ano 1)' : 'VALOR_ANO_1', 
               'sum(Valor OrÃ§amento Total Ano 2)' : 'VALOR_ANO_2', 
               'sum(Valor OrÃ§amento Total Demais Anos)' : 'VALOR_DEMAIS_ANOS',
                'sum(Valor OrÃ§amento Total)' : 'VALOR_TOTAL',
                'sum(OPEX (%))' : 'PERCENTUAL_OPEX',
                '[SPRO] CÃ³digo do projeto' : 'SR', 
                '[SPRO]ProprietÃ¡rio (CÃ³digo do usuÃ¡rio)' : 'MATRICULA_PROPRIETARIO', 
                '[SPRO]ProprietÃ¡rio (UsuÃ¡rio)' : 'NOME_PROPRIETARIO',
                '[SPRO] Tipo de Solicitacao' : 'TIPO_SOLICITACAO',
                '[SPRO]Projeto (Nome do projeto)' : 'NOME_PROJETO',
                '[SPRO] DescriÃ§Ã£o' : 'DESCRICAO_PROJETO'} 
df_ariba_sr = rename_columns(df_ariba_sr, to_rename_1) 

to_replace_values_1 = 'DT_INI' #'[SPRO]Data inicial (Data)' : 'DT_INI', 
df_ariba_sr = replace_values(df_ariba_sr, to_replace_values_1) 

to_replace_values_1 = 'DT_FIM' #[SPRO]Data final (Data)' : 'DT_FIM'
df_ariba_sr = replace_values(df_ariba_sr, to_replace_values_1) 

new_columns_types = ['VALOR_TOTAL', 'VALOR_DEMAIS_ANOS','VALOR_ANO_2', 'VALOR_ANO_1'] 
df_ariba_sr = transform_column_type(df_ariba_sr, new_columns_types) 

df_ariba_sr = transform_column_int(df_ariba_sr, 'PERCENTUAL_OPEX') 

new_columns = ['NomeArquivo.1', 'NomeArquivo.2', 'NomeArquivo.3', 'NomeArquivo.4', 'NomeArquivo.5']
to_split_column = 'NomeArquivo'
df_ariba_sr = split_column(df_ariba_sr, to_split_column, new_columns) 

to_remove_2 = ['NomeArquivo.2', 'NomeArquivo.3', 'NomeArquivo.4', 'NomeArquivo.5', 'NomeArquivo', 'DIRETORIA']
df_ariba_sr = remove_columns(df_ariba_sr, to_remove_2) 

to_rename_2 = {'NomeArquivo.1': 'DATA_REFERENCIA'}
df_ariba_sr = rename_columns (df_ariba_sr, to_rename_2) 

# to_transform_type = ['SR', 'MATRICULA_PROPRIETARIO', 'NOME_PROPRIETARIO', 
#                     '[SPRO] Tipo de Solicitacao', 'STATUS', 'GRUPO_MERCADORIA', 
#                     'DIRETORIA', 'UO_REQUISITANTE', 'MOEDA', 'CONTA_CONTABIL']
# df_ariba_sr = transform_column_text(df_ariba_sr, to_transform_type) #NA TEORIA NÃO PRECISARIA PORQUE ESSAS COLUNAS JÁ SÃO DO TIPO OBJECT

to_transform_type_date = ['DATA_REFERENCIA', 'DT_INI', 'DT_FIM']
df_ariba_sr = transform_column_date (df_ariba_sr, to_transform_type_date) 

to_sort_ascending = 'SR'
df_ariba_sr = sort_ascending (df_ariba_sr, 'SR') 

to_remove_condicional = 'STATUS'
df_ariba_sr = remove_columns_condicional(df_ariba_sr, to_remove_condicional) 



In [9]:
#preparação dos dfs para o merge
#criar coluna CHAVE para df_de_para_uo_cdc
column_1 = 'N1'
#column_2 = 'N2'
#column_3 = 'N3'
#column_4 = 'DETALHAMENTO'
new_column = 'CHAVE'
df_de_para_uo_cdc = create_column_three(df_de_para_uo_cdc, column_1, new_column) #column2, column_3, column_4

#SEPARAR AS DIRETORIAS DE UO REQUISITANTE EM COLUNAS
new_columns_2 = ['N1_sr', 'N2', 'N3', 'N4']
to_split_column_2= 'UO_REQUISITANTE'
df_ariba_sr = split_column_2(df_ariba_sr, to_split_column_2, new_columns_2) 

#AGREGAR COLUNAS N1 E N2 PARA FAZER O MERGE COM A COLUNA CHAVE 
# df_ariba_sr['N1_N2'] = df_ariba_sr['N1'].astype(str).str.strip() + ' - ' + df_ariba_sr['N2'].astype(str).str.strip()

#APAGAR AS COLUNAS N1, N2, N3 E N4 E MANTER A JUNÇÃO DE N1 E N2, APROVEITANDO E APAGANDO TAMBÉM UO_REQUISITANTE QUE SERÁ APAGADA 
#NA ÚLTIMA LINHA DO CÓDIGO EM M
to_remove_3 = ['N2','N3', 'N4', 'UO_REQUISITANTE']
df_ariba_sr = remove_columns(df_ariba_sr, to_remove_3) 

#df_de_para_uo_cdc.head()
#df_ariba_sr.info()
#df_ariba_sr.head()
#df_ariba_sr.columns()
#print(df_ariba_sr.columns)

In [10]:
# df_ariba_sr = rename_columns(df_ariba_sr, {"[PSO]Projeto (CÃ³digo do projeto)":'SR'})
# # [SPRO] CÃ³digo do projeto
# df_ariba_sr.head()

In [11]:
#"CONTA_CONTABIL" in df_contas_third_parties.columns

In [12]:
#"[FormulÃ¡rio de Sourcing (SR)] Conta ContÃ¡bil" in df_ariba_sr.columns

In [13]:
df_sr_tp = pd.merge(df_ariba_sr, df_contas_third_parties, on='CONTA_CONTABIL', how='left', indicator=True) 
#cruza as informações de sr e contas tp, cria uma coluna com a flag 'both' para os valores que se encontram em ambos os dfs
#df_sr_tp.tail()
#print(df_sr_tp.columns)

In [14]:
#"Merged Queries1" = Table.NestedJoin(#"Filtered Rows1", {"UO_REQUISITANTE"}, DE_PARA_UO_CDC, {"CHAVE"}, "DE_PARA_UO_CDC", JoinKind.LeftOuter),
df_sr_tp_de_para = pd.merge(df_sr_tp, df_de_para_uo_cdc, left_on= 'N1_sr', right_on= 'CHAVE', how='left') 
#df_sr_de_para.head()

#"Expanded DE_PARA_UO_CDC" = Table.ExpandTableColumn(#"Merged Queries1", "DE_PARA_UO_CDC", {"CDC"}, {"CDC"})
#Já tenho a coluna CDC aqui

In [15]:
to_reorder = ['SR', 'MATRICULA_PROPRIETARIO', 'NOME_PROPRIETARIO', 'TIPO_SOLICITACAO', 
              'DT_INI', 'DT_FIM', 'STATUS', 'GRUPO_MERCADORIA', 
              'N1_sr', 'MOEDA', 'CONTA_CONTABIL', 'CDC', 
              'PERCENTUAL_OPEX', 'VALOR_ANO_1', 'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS', 
              'VALOR_TOTAL', 'DATA_REFERENCIA']
df_sr_tp_de_para = reorder_columns(df_sr_tp_de_para, to_reorder)

print(df_sr_tp_de_para.columns)
#df_sr_tp_de_para.head()

Index(['SR', 'MATRICULA_PROPRIETARIO', 'NOME_PROPRIETARIO', 'TIPO_SOLICITACAO',
       'DT_INI', 'DT_FIM', 'STATUS', 'GRUPO_MERCADORIA', 'N1_sr', 'MOEDA',
       'CONTA_CONTABIL', 'CDC', 'PERCENTUAL_OPEX', 'VALOR_ANO_1',
       'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS', 'VALOR_TOTAL', 'DATA_REFERENCIA'],
      dtype='object')


In [16]:
df_sr_tp_de_para['VALOR_ORCADO_OPEX'] = (df_sr_tp_de_para['VALOR_TOTAL'] * (df_sr_tp_de_para['PERCENTUAL_OPEX'] / 100)).astype(float).round(2)
#df_sr_tp_de_para.head()
print(df_sr_tp_de_para.columns)

Index(['SR', 'MATRICULA_PROPRIETARIO', 'NOME_PROPRIETARIO', 'TIPO_SOLICITACAO',
       'DT_INI', 'DT_FIM', 'STATUS', 'GRUPO_MERCADORIA', 'N1_sr', 'MOEDA',
       'CONTA_CONTABIL', 'CDC', 'PERCENTUAL_OPEX', 'VALOR_ANO_1',
       'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS', 'VALOR_TOTAL', 'DATA_REFERENCIA',
       'VALOR_ORCADO_OPEX'],
      dtype='object')


In [17]:
#df_sr_de_para['VALOR_TOTAL'].describe()

In [18]:
#PREPARAÇÃO PARA UNION DE TODOS OS DFS
#DF SR TP DE PARA >> MERGE DO SR COM TP E DE PARA
#FALTA COLOCAR A COLUNA COUNT IF QUE EU PRECISO AINDA ENTENDER DO QUE SE TRATA

#"Sorted Rows" = Table.Sort(#"Filtered Rows",{{"SR", Order.Ascending}})
to_sort_ascending = 'SR'
df_sr_tp_de_para = sort_ascending (df_sr_tp_de_para, 'SR') 

#"Removed Columns" = Table.RemoveColumns(#"Sorted Rows",{"VALOR_ANO_1", "VALOR_ANO_2", "VALOR_DEMAIS_ANOS"}),
to_remove_10 = ['VALOR_ANO_1', 'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS']
df_sr_tp_de_para = remove_columns(df_sr_tp_de_para, to_remove_10)

#"Renamed Columns" = Table.RenameColumns(#"Removed Columns",{{"VALOT_TOTAL", "VALOR_ORCADO"}}),
to_rename_7 = {'VALOR_TOTAL': 'VALOR_ORCADO'} 
df_sr_tp_de_para = rename_columns(df_sr_tp_de_para, to_rename_7) 

#"Reordered Columns" = Table.ReorderColumns(#"Renamed Columns",{"SR", 
# "MATRICULA_PROPRIETARIO", "NOME_PROPRIETARIO", "TIPO_SOLICITACAO", 
# "DT_INI", "DT_FIM", "STATUS", "GRUPO_MERCADORIA", "MOEDA", "CONTA_CONTABIL", "CDC", 
# "PERCENTUAL_OPEX", "VALOR_ORCADO", "VALOR_ORCADO_OPEX", "DATA_REFERENCIA", "COUNT_IF"})
to_reorder_4 = ['SR', 'MATRICULA_PROPRIETARIO', 'NOME_PROPRIETARIO',
                'TIPO_SOLICITACAO', 'DT_INI', 'DT_FIM',
                'STATUS', 'GRUPO_MERCADORIA', 'MOEDA',
                'CONTA_CONTABIL', 'CDC', 'PERCENTUAL_OPEX',
                'VALOR_ORCADO', 'VALOR_ORCADO_OPEX', 'DATA_REFERENCIA']
df_sr_tp_de_para = reorder_columns(df_sr_tp_de_para, to_reorder_4)

In [19]:
# #UNION DOS 3 DFS: df_ariba_cw, df_ariba_ws e df_sr_tp_de_para 

# df_sr_ws = pd.merge(df_sr_tp_de_para, df_ariba_ws, on='SR', how='left') 

# #"Renamed Columns" = Table.RenameColumns(#"Expanded WS",{{"WS.WS", "WS.CODIGO"}}),
# # to_rename_8 = {'WS':'WS.CODIGO'}
# # df_sr_ws = rename_columns(df_sr_ws, to_rename_8)


# #"Merged Queries1" = Table.NestedJoin(#"Renamed Columns", {"WS.CODIGO"}, CW, {"WS"}, "CW", JoinKind.LeftOuter),
# df_sr_ws_cw= pd.merge(df_sr_ws, df_ariba_cw, on='SR', how='left') 

# # #"Renamed Columns1" = Table.RenameColumns(#"Expanded CW",{{"CW.CW", "CW.CODIGO"}}),

# to_rename_9 = {'DATA_REFERENCIA_x': 'DATA_REFERENCIA'}
# df_sr_ws_cw = rename_columns(df_sr_ws_cw, to_rename_9)
# #print(df_sr_ws_cw.columns)

# # #"Merged Queries2" = Table.NestedJoin(#"Renamed Columns1", {"SR"}, CW, {"SR"}, "CW", JoinKind.LeftOuter),
# df_sr_ws_cw02 = pd.merge(df_sr_ws_cw, df_ariba_cw, on='SR', how='left')

# to_rename_10 = {'CW_x': 'CW.CODIGO'}
# df_sr_ws_cw02 = rename_columns(df_sr_ws_cw02, to_rename_10)

# print(df_sr_ws_cw02.columns)

# # print(df_ariba_cw.columns)

In [20]:
# # Considerando que você tem esses dataframes:
# # df_sr_tp_de_para (tabela principal)
# # df_ariba_ws
# # df_ariba_cw

# # 1. Primeiro merge: df_sr_tp_de_para + df_ariba_ws (join on 'SR')

# # Definindo colunas que queremos trazer de df_ariba_ws, excluindo as de data para evitar duplicidade
# cols_ws = [col for col in df_ariba_ws.columns if col not in ['SR', 'DATA_INI', 'DATA_FIM', 'DATA_REFERENCIA']]

# df_sr_ws = pd.merge(df_sr_tp_de_para, df_ariba_ws[['SR'] + cols_ws], on='SR', how='left')

# # 2. Segundo merge: resultado anterior + df_ariba_cw (join on 'SR')

# # Também removendo colunas de data para evitar duplicidade
# cols_cw = [col for col in df_ariba_cw.columns if col not in ['SR', 'DATA_INI', 'DATA_FIM', 'DATA_REFERENCIA']]

# df_sr_ws_cw = pd.merge(df_sr_ws, df_ariba_cw[['SR'] + cols_cw], on='SR',how='left')

# # 3. Se quiser manter apenas as colunas de data originais do primeiro dataframe (df_sr_tp_de_para),
# # podemos remover as duplicadas que vieram com sufixos _ws ou _cw

# # for suffix in ['_ws', '_cw']:
# #     for col in ['DATA_INI', 'DATA_FIM', 'DATA_REFERENCIA']:
# #         col_sufixed = f"{col}{suffix}"
# #         if col_sufixed in df_sr_ws_cw.columns:
# #             df_sr_ws_cw.drop(columns=[col_sufixed], inplace=True)

# # Agora df_sr_ws_cw tem as colunas de data sem duplicação

# # 4. Se quiser, pode renomear colunas específicas, por exemplo:

# df_sr_ws_cw.rename(columns={'CW': 'CW.CODIGO'}, inplace=True)

# # Ajuste os nomes conforme sua necessidade

# # Exemplo final: imprimir as colunas para conferir
# print(df_sr_ws_cw.columns.tolist())


In [21]:
df_sr_ws = pd.merge(df_sr_tp_de_para, df_ariba_ws, on='SR', how='left')

df_sr_ws_clean = df_sr_ws[['SR','MATRICULA_PROPRIETARIO','NOME_PROPRIETARIO',         
                        'TIPO_SOLICITACAO','DT_INI','DT_FIM',                  
                        'STATUS','GRUPO_MERCADORIA','MOEDA',
                        'CONTA_CONTABIL','CDC','PERCENTUAL_OPEX', 
                        'VALOR_ORCADO','VALOR_ORCADO_OPEX','DATA_REFERENCIA_x',
                        'WS', 'NOME_FORNECEDOR', 'VALOR_ANO_1', 
                        'VALOR_ANO_2', 'VALOR_DEMAIS_ANOS', 'VALOR_TOTAL']]                  

df_sr_ws_cw= pd.merge(df_sr_ws_clean, df_ariba_cw, on='SR', how='left') 

#df_sr_ws_cw_clean= df_sr_ws_cw[['SR','MATRICULA_PROPRIETARIO','NOME_PROPRIETARIO',
                # 'TIPO_SOLICITACAO','DT_INI','DT_FIM',
                # 'STATUS','GRUPO_MERCADORIA','MOEDA',
                # 'CONTA_CONTABIL','CDC','PERCENTUAL_OPEX',
                # 'VALOR_ORCADO','VALOR_ORCADO_OPEX','DATA_REFERENCIA_x',
                # 'WS_x','DATA_INI','DATA_FIM','UO_COMPRADOR',
                # 'NOME_FORNECEDOR','VALOR_ANO_1','VALOR_ANO_2',
                # 'VALOR_DEMAIS_ANOS','VALOR_TOTAL',
                # 'CW', 'CNPJ_CPF_FORNEC', 'FORNECEDOR', 
                # 'VALOR_CONTRATO', 'CW_RAIZ']]

#df_sr_ws_cw02 = pd.merge(df_sr_ws_cw, df_ariba_cw, on='SR', how='left')

df_sr_ws_cw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229512 entries, 0 to 229511
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   SR                      229512 non-null  object        
 1   MATRICULA_PROPRIETARIO  229512 non-null  object        
 2   NOME_PROPRIETARIO       229512 non-null  object        
 3   TIPO_SOLICITACAO        229491 non-null  object        
 4   DT_INI                  229493 non-null  datetime64[ns]
 5   DT_FIM                  200068 non-null  datetime64[ns]
 6   STATUS                  229512 non-null  object        
 7   GRUPO_MERCADORIA        229512 non-null  object        
 8   MOEDA                   229512 non-null  object        
 9   CONTA_CONTABIL          229512 non-null  object        
 10  CDC                     227726 non-null  object        
 11  PERCENTUAL_OPEX         229512 non-null  int64         
 12  VALOR_ORCADO            229512

In [22]:
#COLUNAS CUSTOMIZADAS >>> CRIAR FUNÇÃO!!!
#"Added Custom" = Table.AddColumn
# (#"Expanded CW1", "CW.CODIGO_new", each if [CW.CODIGO] = null then [CW.CW] else [CW.CODIGO]),
#df_sr_ws_cw02['CW_x_new'] = if df_sr_ws_cw02['CW_x'] == None, then df_sr_ws_cw02['CW_y'] else df_sr_ws_cw02['CW_x'] 

df_sr_ws_cw02['CW_x_new'] = df_sr_ws_cw02['CW.CODIGO'].fillna(df_sr_ws_cw02['CW_y'])

#df_sr_ws_cw02.head() TODOS OS DADOS VAZIOS 

#Added Custom1" = Table.AddColumn(#"Added Custom", "CW.CNPJ_CPF_FORNEC_new", each if [CW.CNPJ_CPF_FORNEC] = null then [CW.CNPJ_CPF_FORNEC.1] else [CW.CNPJ_CPF_FORNEC]),
df_sr_ws_cw02['CW.CNPJ_CPF_FORNEC_new'] = df_sr_ws_cw02['CNPJ_CPF_FORNEC_x'].fillna(df_sr_ws_cw02['CNPJ_CPF_FORNEC_y'])

#"Added Custom2" = Table.AddColumn(#"Added Custom1", "CW.FORNECEDOR_new", each if [CW.FORNECEDOR] = null then [CW.FORNECEDOR.1] else [CW.FORNECEDOR]),
df_sr_ws_cw02['CW.FORNECEDOR_new'] = df_sr_ws_cw02['FORNECEDOR_x'].fillna(df_sr_ws_cw02['FORNECEDOR_y'])

#"Added Custom3" = Table.AddColumn(#"Added Custom2", "CW.VALOR_CONTRATO_new", each if [CW.VALOR_CONTRATO] = null then [CW.VALOR_CONTRATO.1] else [CW.VALOR_CONTRATO]),
df_sr_ws_cw02['CW.VALOR_CONTRATO_new'] = df_sr_ws_cw02['VALOR_CONTRATO_x'].fillna(df_sr_ws_cw02['VALOR_CONTRATO_y'])

NameError: name 'df_sr_ws_cw02' is not defined

In [ ]:
#"Removed Columns" = Table.RemoveColumns(#"Added Custom3",{"CW.CODIGO", "CW.CNPJ_CPF_FORNEC", "CW.FORNECEDOR", 
# "CW.VALOR_CONTRATO", !!!"CW.CW"!!!, "CW.CNPJ_CPF_FORNEC.1", "CW.FORNECEDOR.1", "CW.VALOR_CONTRATO.1", "CW.CW_RAIZ_1"}),

to_remove_11 = ['CW.CODIGO', 'CW_RAIZ_x', 'CNPJ_CPF_FORNEC_x', 'FORNECEDOR_x', 'VALOR_CONTRATO_x', 
'DATA_REFERENCIA_x', 'CW_RAIZ_y','CNPJ_CPF_FORNEC_y', 'FORNECEDOR_y', 'VALOR_CONTRATO_y']
df_sr_ws_cw02 = remove_columns(df_sr_ws_cw02, to_remove_11)

#"Renamed Columns2" = Table.RenameColumns(#"Removed Columns",{{"CW.CODIGO_new", "CW.CODIGO"}, 
# {"CW.CNPJ_CPF_FORNEC_new", "CW.CNPJ_CPF_FORNEC"}, 
# {"CW.FORNECEDOR_new", "CW.FORNECEDOR"}, 
# {"CW.VALOR_CONTRATO_new", "CW.VALOR_CONTRATO"}}),

to_rename_11 = {"CW.x_new": "CW.CODIGO", 'CW.CNPJ_CPF_FORNEC_new':'CW.CNPJ_CPF_FORNEC', 
                'CW.FORNECEDOR_new':'CW.FORNECEDOR', 'CW.VALOR_CONTRATO_new':'CW.VALOR_CONTRATO'}
df_sr_ws_cw02 = rename_columns(df_sr_ws_cw02, to_rename_11)

#"Changed Type" = Table.TransformColumnTypes(#"Renamed Columns2",
# {{"DT_INI", type text}, {"DT_FIM", type text}, {"DATA_REFERENCIA", type text}}),

#'DATA_REFERENCIA', 
column_name = ['DT_INI', 'DT_FIM']
df_sr_ws_cw02 = transform_column_text(df_sr_ws_cw02, column_name)

df_sr_ws_cw02.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229512 entries, 0 to 229511
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   SR                      229512 non-null  object        
 1   MATRICULA_PROPRIETARIO  229512 non-null  object        
 2   NOME_PROPRIETARIO       229512 non-null  object        
 3   TIPO_SOLICITACAO        229491 non-null  object        
 4   DT_INI                  229512 non-null  object        
 5   DT_FIM                  229512 non-null  object        
 6   STATUS                  229512 non-null  object        
 7   GRUPO_MERCADORIA        229512 non-null  object        
 8   MOEDA                   229512 non-null  object        
 9   CONTA_CONTABIL          229512 non-null  object        
 10  CDC                     227726 non-null  object        
 11  PERCENTUAL_OPEX         229512 non-null  int64         
 12  VALOR_ORCADO            229512